### SCFT

SCFT is performed with the loss function approach. The user only needs to do the following:

1) Load in the space group structural files associated with the space groups for which we are interested in finding self-consistent solutions.
2) Specify a copolymer sequence (using one of the several provided representations). Here, a diblock sequence of $f_A = 0.38$ and $\chi N = 17.5$ is given as an example.
3) Run SCFT using a method which reproduces the hyperparameters used in the paper.

The class SCFT is contained in src/scft.py and contains the necessary methods. An easy-to-use method called 'run_scft' is provided replicating the hyperparameters used in the paper. Additional methods and options can be found in the .py file.


In [1]:
import sys
import jax.numpy as jnp
import jax
import os

sys.path.append(os.path.abspath(".."))

from src.space_group import SpaceGroupCollection
from src.sequence import Sequence
from src.scft import SCFT
from src.utils import (
    STRUCTURES_DIR,
    PROJECT_ROOT,
)

jax.config.update("jax_enable_x64", True)  # Enable 64-bit precision

**Load in the space group structure files**

In [ ]:
### All this can be stored in a csv file and read line by line
# Define the files we want to load
space_groups_to_load = [
    "P_-1_48_1_1",
    "p_6_m_m_48_48_1",
    "I_a_-3_d_32_32_32",
]  # Replace with actual list of space group names, e.g., ['P_-1_48_1_1']
sg_info_file = (
    PROJECT_ROOT / "space_group_info.csv"
)  # Load the space group info file. Note, if we want to converge the diblock bcc structure, the initial cell guess for Im-3m must be 2. Modify this in the space_group_info file.

# Set up the space group geometries
# sg_collection contains associated info for each space group. For example, we can access sg_collection.space_groups['P_-1_48_1_1'].cell for the cell parameters
# sg_collection['P_-1_48_1_1'].u0['u_fwd'] access the forward u0 initialization for the space group P_-1_48_1_1
# sg_collection['P_-1_48_1_1'].data is a dictionary that allows access to loaded information files

sg_collection = SpaceGroupCollection.from_files(
    desired_sg=space_groups_to_load,
    sg_info_file=sg_info_file,
    structure_folder=STRUCTURES_DIR,
)

**Set up the sequence details**

In [3]:
chi = 17.5
block_fractions = jnp.array(
    [0.38, 0.62]
)  # Specify the block fractions for each block in the block copolymer. Here, 0.38 for A and 0.62 for B.
block_magnitudes = jnp.array(
    [jnp.sqrt(chi) / 2, -jnp.sqrt(chi) / 2]
)  # Example magnitudes for the blocks, can be adjusted as needed. The square difference is the interaction function to go from this 1D sequence to an interaction matrix

# Stack the block fractions and magnitudes into a single array
f_params = jnp.vstack([block_fractions, block_magnitudes])

# Give the sequence a name. This will be the name saved with the results
seq_name = "diblock_f38_c17.5"
sequence = Sequence.generate(
    name=seq_name,
    basis="linear_basis",
    transform="block_copolymer",
    Ns=125,
    Nbasis=125,
    Np=125,
    Ndim=1,
    f_params=f_params,
)  # Refer to README for details on the parameters

# Initialize the SCFT object
scft = SCFT(space_group=sg_collection, sequence=sequence)

**Run SCFT**

In [ ]:
# Run SCFT with default settings
x_dict = {}
p_dict = {}
sg_list = [
    "P_-1_48_1_1",
    "p_6_m_m_48_48_1",
    "I_a_-3_d_32_32_32",
]  # This is the list of space groups we want to run SCFT on.

for sg in sg_list:
    x_scft, p_scft = scft.run_scft(
        sg=sg, load=False, save=True
    )  # load, load_dict, save are options
    x_dict[sg] = x_scft
    p_dict[sg] = p_scft

# In each x_scft, there are two keys, one for the 'positive' guess and one for the 'negative' guess. They can be accessed with sg + '_0' and sg + '_1' respectively. For example, to access the positive guess for P_-1_48_1_1, use x_dict['P_-1_48_1_1']['P_-1_48_1_1_0']
# p_scft is a dictionary containing some values related to the SCFT run. 'energy' can be used to access the free energy of the structure. 'w_matlab' refers to the loss used in the sequence-SCFT MATLAB method of Xie, O., and Olsen, B.D. Macromolecules 2022 (https://doi.org/10.1021/acs.macromol.2c00721)